#                              CREDIT ANALYTICS

Libraries to load and read data from bigquery

In [16]:
#installing packages
#!pip install db-dtypes
#!pip install pandas_gbq

In [17]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import db_dtypes

> CREATE CONNECTION TO BIGQUERY

In [13]:
# Constants
PROJECT_ID = 'datapro-405709'
KEY_PATH = '/work/datapro-405709-6e4eacf25d80.json'

#Credentials and Client
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

# Display the client details
client

> READ AND LOAD DATA FROM BIGQUERY

In [19]:
# Since 'db-dtypes' package doesn't exist, we do not need to import it. Here is the corrected code.
query = "SELECT * FROM `Credit_Business.Disbursements`"
query_job = client.query(query)
df = query_job.to_dataframe()

ValueError: Please install the 'db-dtypes' package to use this function.

In [ ]:
#print columns
print(df.columns)

Index(['S_N', 'Branch_Code', 'Branch_Name', 'Account_Number', 'Account_Name',
       'Gender', 'Birth_date', 'Product', 'Sector', 'Loan_Cycle', 'Term',
       'Disbursed___Amount', 'Interest_Rate', 'Principal_Outstanding',
       'Interest_Outstanding', 'Grace_Period', 'Due_days', 'Disbursed_Date',
       'Disbursed_By', 'Approved_By', 'Insurance_company', 'Sub_Sector',
       'GROUP___CUSTOMER_NUMBER'],
      dtype='object')


In [ ]:
import pandas as pd

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.2f}'.format

# Drop unnecessary columns, while ensuring they exist first
columns_to_drop = ['Branch_Code', 'Account_Number', 'Account_Name', 'S_N', 'GROUP___CUSTOMER_NUMBER']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Convert data types
df = df.astype({
    'Disbursed_Date': 'datetime64',
    'Disbursed_By': 'string',
    'Approved_By': 'string',
    'Branch_Name': 'string',
    'Insurance_company': 'string',
    'Sub_Sector': 'string',
    'Birth_date': 'datetime64',
    'Due_days': 'int64',
    'Interest_Rate': 'float64',
    'Principal_Outstanding': 'float64',
    'Interest_Outstanding': 'float64',
    'Grace_Period': 'string',
    'Loan_Cycle': 'int64',
    'Term': 'string',
    'Gender': 'string',
    'Product': 'string',
    'Sector': 'string',
    'Disbursed___Amount': 'float64',   
})

# Display the first 10 rows of dataframe
df.head(10)

,Branch_Name,Gender,Birth_date,Product,Sector,Loan_Cycle,Term,Disbursed___Amount,Interest_Rate,Principal_Outstanding,Interest_Outstanding,Grace_Period,Due_days,Disbursed_Date,Disbursed_By,Approved_By,Insurance_company,Sub_Sector
0,Head Office,M,1987-08-08,Staff Salary Advance,Trade,8,6 M,"1,000,000.00",0.00,"1,000,000.00",0.00,<NA>,0,2023-11-10,ODONG ALEX,Yahaya Ochaya,<NA>,Exports
1,Head Office,M,1992-05-02,Staff Salary Advance,Personal Loans and Household Loans,8,6 M,"2,700,000.00",0.00,"2,700,000.00",0.00,<NA>,0,2023-11-07,ODONG ALEX,Solomon Mwesigwa,<NA>,Personal Loans and Household Loans
2,Head Office,M,1987-02-07,Staff Salary Advance,Personal Loans and Household Loans,3,6 M,"4,000,000.00",0.00,"4,000,000.00",0.00,<NA>,0,2023-11-06,ODONG ALEX,Henry Bukenya,<NA>,Personal Loans and Household Loans
3,Head Office,F,1990-03-03,Staff Salary Advance,Personal Loans and Household Loans,4,6 M,"7,500,000.00",0.00,"7,500,000.00",0.00,<NA>,0,2023-11-01,NAMIREMBE MILLY,Jimmy Kasoro,<NA>,Personal Loans and Household Loans
4,Head Office,M,1990-04-10,Staff Salary Advance,Personal Loans and Household Loans,2,6 M,"3,100,000.00",0.00,"3,100,000.00",0.00,<NA>,0,2023-11-10,ODONG ALEX,Yahaya Ochaya,<NA>,Personal Loans and Household Loans
5,Head Office,M,1986-12-10,Staff Salary Advance,Personal Loans and Household Loans,2,5 M,"6,000,000.00",0.00,"6,000,000.00",0.00,<NA>,0,2023-11-07,ODONG ALEX,Yahaya Ochaya,<NA>,Personal Loans and Household Loans
6,Head Office,M,1995-03-27,Staff Salary Advance,Personal Loans and Household Loans,1,6 M,"2,500,000.00",0.00,"2,500,000.00",0.00,<NA>,0,2023-11-07,ODONG ALEX,Solomon Mwesigwa,<NA>,Personal Loans and Household Loans
7,Head Office,F,1985-01-16,Staff Salary Advance,Personal Loans and Household Loans,10,6 M,"1,800,000.00",0.00,"1,800,000.00",0.00,<NA>,0,2023-11-03,ODONG ALEX,Jimmy Kasoro,<NA>,Personal Loans and Household Loans
8,Head Office,F,1987-01-10,Staff Salary Advance,Personal Loans and Household Loans,16,6 M,"9,900,000.00",0.00,"9,900,000.00",0.00,<NA>,0,2023-11-08,ODONG ALEX,Solomon Mwesigwa,<NA>,Personal Loans and Household Loans
9,Head Office,M,1969-11-02,Staff Salary Advance,Personal Loans and Household Loans,9,6 M,"10,200,000.00",0.00,"10,200,000.00",0.00,<NA>,0,2023-11-03,ODONG ALEX,Jimmy Kasoro,<NA>,Personal Loans and Household Loans


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=34c99b87-7747-401d-b203-ddc5eb2646f3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>